# App data analysis

    In this project, we are interested in coverting the functional solution from the 'Data_Analysis_Project' file to an object oriented solution.

In [18]:
from csv import reader


class DataManager():
    ### reads Google Play's dataset
    def handle_data(self, dataset):
        opened_file = open(dataset, encoding="utf8")
        read_file = reader(opened_file)
        data = list(read_file)
        self.data_header = data[0]
        self.dataset = data[1:]
        
    ## Function that prints dataset in a pretty way
    def explore_data(self, start, end, rows_and_columns=False):
        dataset_slice = self.dataset[start:end]    
        for row in dataset_slice:
            print(row)
            print('\n') # adds a new (empty) line after each row

        if rows_and_columns:
            print('Number of rows:', len(dataset))
            print('Number of columns:', len(dataset[0]))
    
DM = DataManager()
DM.handle_data('googleplaystore.csv')
print(DM.data_header)

DM2 = DataManager()
DM2.handle_data('AppleStore.csv')
print(DM2.data_header)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


In [19]:
DM.explore_data(0, 4, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']




NameError: name 'dataset' is not defined

In [7]:
explore_data(ios_data, 0, 4, True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


Number of rows: 7197
Number of columns: 16


Looking at the Google Play dataset's [*Discussions*](https://www.kaggle.com/lava18/google-play-store-apps/discussion) some problems have come to my attention. There are:
    - A wrong entrie on the 10472 row
    - Multiple entries for the same app:

In [8]:
print(ios_header)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


In [5]:
print(android_header)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [11]:
print(android_data[10472])

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


In [13]:
print(android_data[10473])

['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


In [3]:
del android_data[10472]

Let me show you an exemple of multiple entries:

In [11]:
unique_app_names = []
duplicate_app_names = {}

for app in android_data:
    app_name = app[0]
    if not app_name in unique_app_names:
        unique_app_names.append(app_name)
    elif not app_name in duplicate_app_names:
        duplicate_app_names[app_name] = 1
    else:
        duplicate_app_names[app_name] += 1
unique_app_amount = len(unique_app_names)
total_app_amount = len(android_data)

print("Amount of unique apps " + str(unique_app_amount))
print("Amount of duplicate entries: " + str(total_app_amount - unique_app_amount))


Amount of unique apps 9660
Amount of duplicate entries: 1181


To remove the duplicate entries, we are going to consider which entry has the most reviews and delete the rest. The reason for that is the entry with most reviews is likely to be the most recent one.

In [9]:
reviews_max = {}

for app in android_data:
    name = app[0]
    n_reviews = float(app[3])
    if (name in reviews_max) and (reviews_max[name] < n_reviews):
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
        
print(len(reviews_max))    

9659


In [11]:
android_clean = []
already_added = []

for app in android_data:
    name = app[0]
    n_reviews = float(app[3])
    
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(app)
        already_added.append(name)
        
print(android_clean[:5])

[['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'], ['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'], ['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'], ['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up'], ['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']]


We are interested in apps for english speaking users, so we are going to exclude apps that have non-english characters in their names such as '爱'

In [16]:
def is_english_name(name):
    excentric_characters = 0
    for character in name:
        if ord(character) > 127:
            return False
    return True

#testing function
print(is_english_name('Instagram'))
print(is_english_name('爱奇艺PPS -《欢乐颂2》视剧热播'))
print(is_english_name('Docs To Go™ Free Office Suite'))
print(is_english_name('Instachat 😜'))

True
False
False
False


Above we identified that some english titles didn't pass our test because they have a character greater than 127. A way to minimize this is to tolerate at most 3 characters greater than 127.

In [17]:
def is_english_name(name):
    excentric_characters = 0
    for character in name:
        if ord(character) > 127:
            excentric_characters += 1
    if excentric_characters > 2:
        return False
    return True

#testing function
print(is_english_name('Instagram'))
print(is_english_name('爱奇艺PPS -《欢乐颂2》视剧热播'))
print(is_english_name('Docs To Go™ Free Office Suite'))
print(is_english_name('Instachat 😜'))

True
False
True
True


Now that we have a satisfactory filter for non-english apps, let's filter our datasets.

In [23]:
android_cleaner = []
ios_clean = []

for app in android_clean:
    name = app[0]
    if is_english_name(name):
        android_cleaner.append(app)
        
print(len(android_cleaner))

for app in ios_data:
    name = app[1]
    if is_english_name(name):
        ios_clean.append(app)

print(len(ios_clean))

9597
6155


Since we are interested in analysing the free apps, we are going to filter out the paid apps

In [30]:
android_free = []
ios_free = []

for app in android_cleaner:
    price = app[6]
    if price == 'Free':
        android_free.append(app)
        
print(len(android_free))

for app in ios_clean:
    price = app[4]
    if price == '0.0':
        ios_free.append(app)

print(len(ios_free))

8847
3203


To validade our app idea, we want to find a genre in both ios and android that attract the most users.

In [36]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])
        
def freq_table(dataset, index):
    frequency = {}
    for row in dataset:
        column_data = row[index]
        if column_data in frequency:
            frequency[column_data] += 1
        else:
            frequency[column_data] = 1
    for genre in frequency:
        frequency[genre] = round((frequency[genre]/len(dataset)) * 100, 2)
    return frequency


display_table(ios_free, 11)

Games : 58.26
Entertainment : 7.84
Photo & Video : 5.0
Education : 3.68
Social Networking : 3.31
Shopping : 2.59
Utilities : 2.47
Sports : 2.15
Music : 2.06
Health & Fitness : 2.03
Productivity : 1.75
Lifestyle : 1.56
News : 1.34
Travel : 1.25
Finance : 1.09
Weather : 0.87
Food & Drink : 0.81
Reference : 0.53
Business : 0.53
Book : 0.37
Navigation : 0.19
Medical : 0.19
Catalogs : 0.12


In [38]:
display_table(android_free, 1)
print('\n')
display_table(android_free, 9)

FAMILY : 18.93
GAME : 9.7
TOOLS : 8.45
BUSINESS : 4.6
PRODUCTIVITY : 3.9
LIFESTYLE : 3.89
FINANCE : 3.71
MEDICAL : 3.54
SPORTS : 3.39
PERSONALIZATION : 3.32
COMMUNICATION : 3.23
HEALTH_AND_FITNESS : 3.09
PHOTOGRAPHY : 2.95
NEWS_AND_MAGAZINES : 2.8
SOCIAL : 2.67
TRAVEL_AND_LOCAL : 2.34
SHOPPING : 2.25
BOOKS_AND_REFERENCE : 2.14
DATING : 1.87
VIDEO_PLAYERS : 1.8
MAPS_AND_NAVIGATION : 1.39
FOOD_AND_DRINK : 1.24
EDUCATION : 1.16
ENTERTAINMENT : 0.96
LIBRARIES_AND_DEMO : 0.94
AUTO_AND_VEHICLES : 0.93
HOUSE_AND_HOME : 0.8
WEATHER : 0.79
EVENTS : 0.71
PARENTING : 0.66
ART_AND_DESIGN : 0.64
COMICS : 0.61
BEAUTY : 0.6


Tools : 8.44
Entertainment : 6.08
Education : 5.36
Business : 4.6
Productivity : 3.9
Lifestyle : 3.88
Finance : 3.71
Medical : 3.54
Sports : 3.46
Personalization : 3.32
Communication : 3.23
Action : 3.1
Health & Fitness : 3.09
Photography : 2.95
News & Magazines : 2.8
Social : 2.67
Travel & Local : 2.33
Shopping : 2.25
Books & Reference : 2.14
Simulation : 2.05
Dating : 1.87
Arc

In [42]:
prime_genre_freq = freq_table(ios_free, 11)

for genre in prime_genre_freq:
    total = 0
    len_genre = 0
    for row in ios_free:
        genre_app = row[11]
        if genre_app == genre:
            user_rating_total = float(row[5])
            total += user_rating_total
            len_genre += 1
    average = total/len_genre
    print("App genre: " + str(genre) + " Average rating: " + str(round(average, 2)))
            

App genre: Social Networking Average rating: 71548.35
App genre: Photo & Video Average rating: 28441.54
App genre: Games Average rating: 22886.37
App genre: Music Average rating: 57326.53
App genre: Reference Average rating: 79350.47
App genre: Health & Fitness Average rating: 23298.02
App genre: Weather Average rating: 52279.89
App genre: Utilities Average rating: 19156.49
App genre: Travel Average rating: 28243.8
App genre: Shopping Average rating: 27230.73
App genre: News Average rating: 21248.02
App genre: Navigation Average rating: 86090.33
App genre: Lifestyle Average rating: 16815.48
App genre: Entertainment Average rating: 14195.36
App genre: Food & Drink Average rating: 33333.92
App genre: Sports Average rating: 23008.9
App genre: Book Average rating: 46384.92
App genre: Finance Average rating: 32367.03
App genre: Education Average rating: 7003.98
App genre: Productivity Average rating: 21028.41
App genre: Business Average rating: 7491.12
App genre: Catalogs Average rating: 40

In [45]:
genre_freq = freq_table(android_free, 1)

for category in genre_freq:
    total = 0
    len_category = 0 
    for row in android_free:
        category_app = row[1]
        if category_app == category:
            installs = row[5]
            clean_installs = float(installs.replace('+','').replace(',',''))
            total += clean_installs
            len_category += 1
            
    average_android = total/len_category
    print("App genre: " + str(category) + " Average rating: " + str(round(average_android, 2)))

App genre: ART_AND_DESIGN Average rating: 1986335.09
App genre: AUTO_AND_VEHICLES Average rating: 647317.82
App genre: BEAUTY Average rating: 513151.89
App genre: BOOKS_AND_REFERENCE Average rating: 8814199.79
App genre: BUSINESS Average rating: 1712290.15
App genre: COMICS Average rating: 832613.89
App genre: COMMUNICATION Average rating: 38590581.09
App genre: DATING Average rating: 854028.83
App genre: EDUCATION Average rating: 1833495.15
App genre: ENTERTAINMENT Average rating: 11640705.88
App genre: EVENTS Average rating: 253542.22
App genre: FINANCE Average rating: 1387692.48
App genre: FOOD_AND_DRINK Average rating: 1924897.74
App genre: HEALTH_AND_FITNESS Average rating: 4188821.99
App genre: HOUSE_AND_HOME Average rating: 1360598.04
App genre: LIBRARIES_AND_DEMO Average rating: 638503.73
App genre: LIFESTYLE Average rating: 1446158.22
App genre: GAME Average rating: 15544014.51
App genre: FAMILY Average rating: 3697848.17
App genre: MEDICAL Average rating: 120550.62
App genre: